In [59]:
# !pip install langgraph crewai langchain faiss-cpu


In [60]:
# !pip install --upgrade litellm

In [ ]:
from langchain.vectorstores import FAISS
from langgraph.graph import StateGraph, END
from crewai import Agent, Task, Crew
from typing import Dict, Optional
from dataclasses import dataclass
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from litellm import completion
import os
from dotenv import load_dotenv
from langchain_community.llms import HuggingFaceEndpoint
from litellm import completion
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")


## Defining the LLM inference model: https://docs.litellm.ai/docs/providers/huggingface
llm = HuggingFaceEndpoint( 
    repo_id="huggingface/together/deepseek-ai/DeepSeek-R1",
    huggingfacehub_api_token=os.getenv("HF_TOKEN"),
    task="text-generation",
)

# Initialize Embeddings (open source)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



loader=TextLoader("DSA.md")
documents=loader.load()



text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs=text_splitter.split_documents(documents)

db=FAISS.from_documents(docs,embeddings)
db.save_local("faiss_index")

# Load FAISS database for knowledge retrieval
vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()

# Define State class
@dataclass
class PrepAIState:
    question: Optional[str] = ""
    retrieved_info: Optional[str] = ""
    final_response: Optional[str] = ""

# Function to retrieve relevant knowledge from FAISS
def retrieve_knowledge(state: PrepAIState) -> Dict:
    question = state.question
    docs = retriever.get_relevant_documents(question)
    retrieved_info = "\n".join([doc.page_content for doc in docs])
    return {"retrieved_info": retrieved_info}

# Define the DSA AI Agent
dsa_agent = Agent(
    role="DSA AI",
    goal="Evaluate data structures and algorithms solutions",
    backstory="An expert in competitive programming and algorithm optimization.",
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=llm 
    
)

# Define the LLD AI Agent
lld_agent = Agent(
    role="Low-Level Design AI",
    goal="Review system design responses",
    backstory="A highly skilled software architect evaluating system design decisions.",
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=llm 
)

# Define the HLD AI Agent
hld_agent = Agent(
    role="High-Level Design AI",
    goal="Provide feedback on system architecture",
    backstory="An experienced cloud architect reviewing scalability and performance aspects.",
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=llm 
)

# Function to execute AI agent tasks
def crewai_task(state: PrepAIState) -> Dict:
    retrieved_info = state.retrieved_info

    dsa_task = Task(
        description=f"Analyze the following algorithm question and provide an optimized solution:\n\n{retrieved_info}",
        expected_output="An optimized algorithmic solution with time and space complexity analysis.",
        agent=dsa_agent,
    )

    lld_task = Task(
        description=f"Analyze the following system design question and provide feedback:\n\n{retrieved_info}",
        expected_output="A detailed review of the low-level design, including data flows, class diagrams, and possible improvements.",
        agent=lld_agent,
    )

    hld_task = Task(
        description=f"Review the high-level architecture of the following question:\n\n{retrieved_info}",
        expected_output="Scalability, availability, and performance suggestions with architecture diagrams or principles.",
        agent=hld_agent,
    )

    crew = Crew(agents=[dsa_agent, lld_agent, hld_agent], tasks=[dsa_task, lld_task, hld_task])
    results = crew.kickoff()

    return {"final_response": "\n".join(results)}

# Define execution graph using LangGraph
workflow = StateGraph(PrepAIState)
workflow.add_node("retrieve", retrieve_knowledge)
workflow.add_node("crew_analysis", crewai_task)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "crew_analysis")
workflow.add_edge("crew_analysis", END)

app = workflow.compile()


# Example usage
question = "How to find fibonacci series in most optimised time?"
inputs = {"question": question}
result = app.invoke(inputs)

print(result)